In [11]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import matt_prepare
import visualization
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

---

**Down the rabbit hole - trying to scrape 10,000 repository readme files**

Since the models we created saw significant drop off in performance likely due to the small size of our original dataset I wanted to try to pull a bigger set of repo readme files and started the journey in this notebook. I copy and pasted the important code cells to the top of this notebook in case anyone wanted to look them over. The rest of the notebook is pretty messy from the debugging process. In the end I was able to create a .json with 2000 readme files right before the project due date/time so they didnt make it into the project but I did run a copy of our notebook with the new data set and saw some improvement in model performance but not as much as I had expected. 

Takeaways:
- GitHub API will limit you to 5000 requests per hour which made it challenging to pull as many readme files as I wanted, I got limited multiple times and needed the code below to see how long I would be limited for before I could make more requests

- The first time I ran the acquire.py file I had no idea if it was working or not or how long it needed to run so we went back and added print statements to let me know where the function was at. These included:
    - print(f'error making request {url}') in the github_api_request function
    
    - For the scrape_github_data we changed to this instead of the list comp
    
    output = []
    
    for i, repo in enumerate(REPOS):
        
        print(f'\r {i} / {len(REPOS)}', end='')
        
        output.append(process_repo(repo))
    
    return output

- There were a few repo names that had no readme or were an empty repo so they would break the loop. The above additions to the acquire.py file help locate whch ones they were so they could be removed from the list

- In the end I had to split the list of 10,000 repos into 5 groups of 2000 to get it to run and only got through pulling one set of 2000 but not with enough time to make it into the project.

- I did run a copy of our project notebook with the set of 2000 readme files and saw some increased performance but not quite as much as I expected

In [ ]:
#this cell pulls 1000 usernam/repo name combos from the github api based on the entered searchword and 
#saves them localy as a .csv the search word and filename for the csv need to be changed each time this cell is ran 
#so you are not pulling the same repos and so the previous csv is not overwritten.
 
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

params = {
    'sort': 'stars',
    'per_page': 100,
    'q': 'predict' #change the query / searchword here for each 1000 repos pulled
}

repos = []

max_page = 10

for page in range(1, max_page + 1):
    print(f'Getting page [{page}/{max_page}]')
    params['page'] = page
    response = requests.get('https://api.github.com/search/repositories', headers=headers, params=params)
    items = response.json()['items']
    urls = [item['full_name'] for item in items]
    repos.extend(urls)

pd.Series(repos).rename('repo_name').to_csv('repos_predict.csv', index=False) #change the name of the output .csv file
                                                                            # each time you run the cell otherwise the
                                                                            #it will overwrite the previous list

In [12]:
##### read in the 10 .csv files with a list of 1000 repos and concatenate into one df ######

df1 = pd.read_csv('repos_predict.csv')
df2 = pd.read_csv('repos_statistics.csv')
df3 = pd.read_csv('repos_tidy_data.csv')
df4 = pd.read_csv('repos_machine_learning.csv')
df5 = pd.read_csv('repos_data_viz.csv')
df6 = pd.read_csv('repos_modeling.csv')
df7 = pd.read_csv('repos_learn_to_code.csv')
df8 = pd.read_csv('repos_programming.csv')
df9 = pd.read_csv('repos_data_science.csv')
df10 = pd.read_csv('repos.csv')
repo_list = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=0)
repo_list.shape

(10000, 1)

In [13]:
#the following code changes the df into a list of strings so it can be used with the acquire.py file zach gave us#
repo_list.repo_name.values.tolist()

['apache/predictionio',
 'huseinzol05/Stock-Prediction-Models',
 'kootenpv/whereami',
 'jaungiers/LSTM-Neural-Network-for-Time-Series-Prediction',
 'borisbanushev/stockpredictionai',
 'samyk/magspoof',
 'quantopian/alphalens',
 'minimaxir/automl-gs',
 'mathisonian/premonish',
 'idealo/image-quality-assessment',
 'VivekPa/AIAlpha',
 'shirosaidev/stocksight',
 'lilianweng/stock-rnn',
 'mikebryant/ac-nh-turnip-prices',
 'babybuddy/babybuddy',
 'uncertainty-toolbox/uncertainty-toolbox',
 'shobrook/BitVision',
 'iro-cp/FCRN-DepthPrediction',
 'TIBCOSoftware/snappydata',
 'guillaume-chevalier/seq2seq-signal-prediction',
 'jiachenli94/Awesome-Interaction-aware-Trajectory-Prediction',
 'scorpionhiccup/StockPricePrediction',
 'ethz-adrl/control-toolbox',
 'qiaoguan/deep-ctr-prediction',
 '8080labs/ppscore',
 'llSourcell/How-to-Predict-Stock-Prices-Easily-Demo',
 'xuehaouwa/Awesome-Trajectory-Prediction',
 'csinva/imodels',
 'Johnson0722/CTR_Prediction',
 'CalculatedContent/WeightWatcher',
 'cox

**Use the above code assigned as the 'REPOS' variable in the acquire.py file shown below to pull the readme files**

In [ ]:
"""
A module for obtaining repo readme and language data from the github API.

Before using this module, read through it, and follow the instructions marked
TODO.

After doing so, run it like this:

    python acquire.py

To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = repo_list.repo_name.values.tolist()

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        print(f'error making request {url}')#prints out what url / repo that the loop broke on so you can investigate
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text if readme_download_url is not None else ""
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    output = []
    for i, repo in enumerate(REPOS):
        print(f'\r {i} / {len(REPOS)}', end='')
        output.append(process_repo(repo))
    return output

    
    #return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)


**The above functions from zach's acquire.py file is trying to pull the readme and languages from the 10,000 repos in the list and each repo makes 2 requests to github for a total of 20,000 requests. The GitHub API limits you to 5000 requests per hour (2500 repo readme pulls) so I got limited multiple times and ended up splitting the list back up as seen below**

In [ ]:
#split the repo list (df with one column containing 10,000 repo names) into five chunks 
#so you wont get limited by the github api
df1 = repo_list.iloc[:2000]
df2 = repo_list.iloc[2001:4000]
df3 = repo_list.iloc[4001:6000]
df4 = repo_list.iloc[6001:8000]
df5 = repo_list.iloc[8001:9999]

In [77]:
#verify the split
df1.shape, df2.shape, df3.shape, df4.shape, df5.shape

((2000, 1), (1998, 1), (1999, 1), (1999, 1), (1998, 1))

In [ ]:
#this code can be used to push the split list of repo names through the acquire.py function 
#to pull 2000 repos at a time (4000 requests) and not get limited by the github api
#as with the code above to pull the lists of repos this will have to be done in stages and you need to rename 
#the .json file output at the bottom of the .py function so the previous list is not overwritten
df1.repo_name.values.tolist()

**If you do get limited by GitHub you can use the following code to see how long you need to wait before you will be able to make requests again**

In [78]:
import requests
from env import github_token, github_username
from time import time

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

url = 'https://api.github.com/rate_limit'
response = requests.get(url, headers=headers)
my_reset_time = response.json()['resources']['core']['reset']
seconds_until_reset = my_reset_time - time()
minutes_until_reset = seconds_until_reset / 60

In [79]:
minutes_until_reset

44.68882513443629

---

---

---

In [22]:
repo_list.repo_name.values[2467]

'austin-was-taken/Getting-and-Cleaning-Data-Course-Project'

In [46]:
process_repo('austin-was-taken/Getting-and-Cleaning-Data-Course-Project')

{'repo': 'austin-was-taken/Getting-and-Cleaning-Data-Course-Project',
 'language': 'R',
 'readme_contents': '# Getting-and-Cleaning-Data-Course-Project\n## Project Statement\nThe purpose of this project is to demonstrate your ability to collect, work with, and clean a data set. The goal is to prepare tidy data that can be used for later analysis. You will be graded by your peers on a series of yes/no questions related to the project. You will be required to submit: 1) a tidy data set as described below, 2) a link to a Github repository with your script for performing the analysis, and 3) a code book that describes the variables, the data, and any transformations or work that you performed to clean up the data called CodeBook.md. You should also include a README.md in the repo with your scripts. This repo explains how all of the scripts work and how they are connected.    \n\nOne of the most exciting areas in all of data science right now is wearable computing - see for example this art

In [29]:
repo_list.reset_index(drop=True).drop(index=2467)


,repo_name
0,apache/predictionio
1,huseinzol05/Stock-Prediction-Models
2,kootenpv/whereami
3,jaungiers/LSTM-Neural-Network-for-Time-Series-...
4,borisbanushev/stockpredictionai
...,...
9995,jbhuang0604/awesome-computer-vision
9996,matryer/xbar
9997,bevyengine/bevy
9998,nondanee/UnblockNeteaseMusic


In [42]:
repo_list.repo_name.values[2476]

'waleedaldweesh/Wrangle-and-Analyze-Data'

In [44]:
repo_list

,repo_name
0,apache/predictionio
1,huseinzol05/Stock-Prediction-Models
2,kootenpv/whereami
3,jaungiers/LSTM-Neural-Network-for-Time-Series-...
4,borisbanushev/stockpredictionai
...,...
995,jbhuang0604/awesome-computer-vision
996,matryer/xbar
997,bevyengine/bevy
998,nondanee/UnblockNeteaseMusic


In [47]:
repo_list = repo_list.reset_index(drop=True).drop(index=2476)

In [40]:
"""
A module for obtaining repo readme and language data from the github API.

Before using this module, read through it, and follow the instructions marked
TODO.

After doing so, run it like this:

    python acquire.py

To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = repo_list.repo_name.values.tolist()

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        print(f'error making request {url}')
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text if readme_download_url is not None else ""
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    output = []
    for i, repo in enumerate(REPOS):
        print(f'\r {i} / {len(REPOS)}', end='')
        output.append(process_repo(repo))
    return output

    
    #return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)


 2476 / 10000error making request https://api.github.com/repos/waleedaldweesh/Wrangle-and-Analyze-Data/contents/


Exception: Error response from github api! status code: 404, response: {"message": "This repository is empty.", "documentation_url": "https://docs.github.com/v3/repos/contents/#get-contents"}

In [ ]:
!mv data.json data-original.json

In [9]:
repo_list.repo_name.isnull().sum()

0

In [31]:
url = 'https://api.github.com/api/v3/rate_limit'
response = requests.get(url, headers=headers)
response.json()

{'message': 'API rate limit exceeded for user ID 96085443.',
 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}

In [33]:
from time import time
my_reset_time = ...
seconds_until_rate_limit_reset = my_reset_time - time()

TypeError: unsupported operand type(s) for -: 'ellipsis' and 'float'

In [74]:
import requests
from env import github_token, github_username
from time import time

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

url = 'https://api.github.com/rate_limit'
response = requests.get(url, headers=headers)
my_reset_time = response.json()['resources']['core']['reset']
seconds_until_reset = my_reset_time - time()
minutes_until_reset = seconds_until_reset / 60

In [75]:
minutes_until_reset

7.871795384089152

In [50]:
repo_list.shape


(9999, 1)

In [60]:
df1 = repo_list.iloc[:2000]
df2 = repo_list.iloc[2001:4000]
df3 = repo_list.iloc[4001:6000]
df4 = repo_list.iloc[6001:8000]
df5 = repo_list.iloc[8001:9999]

In [61]:
df1.shape, df2.shape, df3.shape, df4.shape, df5.shape

((2000, 1), (1999, 1), (1999, 1), (1999, 1), (1998, 1))

In [62]:
df1.repo_name.values.tolist()

['apache/predictionio',
 'huseinzol05/Stock-Prediction-Models',
 'kootenpv/whereami',
 'jaungiers/LSTM-Neural-Network-for-Time-Series-Prediction',
 'borisbanushev/stockpredictionai',
 'samyk/magspoof',
 'quantopian/alphalens',
 'minimaxir/automl-gs',
 'mathisonian/premonish',
 'idealo/image-quality-assessment',
 'VivekPa/AIAlpha',
 'shirosaidev/stocksight',
 'lilianweng/stock-rnn',
 'mikebryant/ac-nh-turnip-prices',
 'babybuddy/babybuddy',
 'uncertainty-toolbox/uncertainty-toolbox',
 'shobrook/BitVision',
 'iro-cp/FCRN-DepthPrediction',
 'TIBCOSoftware/snappydata',
 'guillaume-chevalier/seq2seq-signal-prediction',
 'jiachenli94/Awesome-Interaction-aware-Trajectory-Prediction',
 'scorpionhiccup/StockPricePrediction',
 'ethz-adrl/control-toolbox',
 'qiaoguan/deep-ctr-prediction',
 '8080labs/ppscore',
 'llSourcell/How-to-Predict-Stock-Prices-Easily-Demo',
 'xuehaouwa/Awesome-Trajectory-Prediction',
 'csinva/imodels',
 'Johnson0722/CTR_Prediction',
 'CalculatedContent/WeightWatcher',
 'cox

In [67]:
"""
A module for obtaining repo readme and language data from the github API.

Before using this module, read through it, and follow the instructions marked
TODO.

After doing so, run it like this:

    python acquire.py

To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = df1.repo_name.values.tolist()

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        print(f'error making request {url}')
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text if readme_download_url is not None else ""
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    output = []
    for i, repo in enumerate(REPOS):
        print(f'\r {i} / {len(REPOS)}', end='')
        output.append(process_repo(repo))
    return output

    
    #return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("repo_list_1.json", "w"), indent=1)


 1999 / 2000

In [76]:
"""
A module for obtaining repo readme and language data from the github API.

Before using this module, read through it, and follow the instructions marked
TODO.

After doing so, run it like this:

    python acquire.py

To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token: https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = df2.repo_name.values.tolist()

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        print(f'error making request {url}')
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text if readme_download_url is not None else ""
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    output = []
    for i, repo in enumerate(REPOS):
        print(f'\r {i} / {len(REPOS)}', end='')
        output.append(process_repo(repo))
    return output

    
    #return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("repo_list_2.json", "w"), indent=1)


 524 / 1998error making request https://api.github.com/repos/suramrit/tidy/contents/


Exception: Error response from github api! status code: 404, response: {"message": "This repository is empty.", "documentation_url": "https://docs.github.com/v3/repos/contents/#get-contents"}

In [69]:
df2.repo_name.values[483]

'chrisnusa/RandomUserContactsApp'

In [70]:
df2 = df2.reset_index(drop=True).drop(index=483)